In [2]:
import pandas as pd
import numpy as np
import math

#### Read Mutant Original Data

In [3]:
eval_dir = "gc_imdb"
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=["label", "mutant", "gender", "template", "name", "template_id"])
df

,label,mutant,gender,template,name,template_id
0,1,"I have only see three episodes of Hack, starri...",template,"I have only see three episodes of Hack, starri...",template,1846
1,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Roen,1846
2,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Jeet,1846
3,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Hagen,1846
4,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",Willow,1846
...,...,...,...,...,...,...
1891808,1,"First, I'm a huge Dódi fan. I grew up knowing ...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Dódi,1128
1891809,1,"First, I'm a huge Iboyka fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Iboyka,1128
1891810,1,"First, I'm a huge Zsófia fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Zsófia,1128
1891811,1,"First, I'm a huge Ágota fan. I grew up knowing...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",Ágota,1128


#### Read Mutant Prediction

In [4]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [5]:
output_dir = "gc_imdb"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

1891813


In [6]:
df["prediction"] = pred

#### Separate the Original Text with Mutant Text

In [7]:
dft = df[df["gender"] == "template"]
df = df[df["gender"] != "template"]

# dft = df[df.index % 311 == 0]
# df = df[df.index % 311 != 0]

## template
dft = dft.reset_index(drop=True)

## mutant
df = df.reset_index(drop=True)

In [8]:
## template only


dft = dft.drop(columns=["mutant", "gender"])

# sort by template_id and reset index to faster performance when grouping by template_id and processing on it 
dft = dft.sort_values(["template_id"])
dft = dft.reset_index(drop=True)
dft.head()

,label,template,name,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,template,0,0
1,1,Headlines warn us of the current campaign to ...,template,1,1
2,1,"Heather Graham is not just a pretty face,she ...",template,2,1
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",template,3,1
4,0,I understand that people have different expec...,template,4,0


In [9]:
# mutant only

df = df.sort_values(["template_id", "gender"])
df = df.reset_index(drop=True)
df.head()

,label,mutant,gender,template,name,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Sárinka,0,0
1,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Elenka,0,0
2,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Leona,0,0
3,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Katuška,0,0
4,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,Miladka,0,0


## Get Number of Discordant Pairs for Each Template

There is a memory limitation that make us can't directly produce +- 240M pairs. Fortunately, the number of discordant pairs for each template can be calculate theoritically without crossing th data to get +- 240M pairs. This will solve the memory issue.

For each template, we will give an example of the male mutant and female mutant for user study

In [10]:
gb = df.groupby("template_id")
gb.count()

,label,mutant,gender,template,name,prediction
template_id,,,,,,
0,310,310,310,310,310,310
1,310,310,310,310,310,310
2,310,310,310,310,310,310
3,310,310,310,310,310,310
4,310,310,310,310,310,310
...,...,...,...,...,...,...
6078,310,310,310,310,310,310
6079,310,310,310,310,310,310
6080,310,310,310,310,310,310


## Create Rank based on the name on the false positive template

#### Get false positive template 

In [218]:
df_fp = pd.read_csv("../data/dp/labelled-full.csv", encoding="latin1")
df_fp = df_fp.rename(columns={"text": "mutant"})

In [219]:
df_fp

,random_index,sorted_index,template_id,gender,mutant,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
3,540,3,1681,t,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
4,3,4,1681,m,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
...,...,...,...,...,...,...,...,...
694,147,694,1393,m,"Hilary was great as julie, and Pat was once ag...",Yes,Positive,NaN
695,250,695,1393,f,"Hilary was great as julie, and Pat was once ag...",No,Positive,"Mr. Miyagi, Lotte was ""her"" supposed to be his"
696,554,696,228,t,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN
697,698,697,228,m,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN


In [220]:
# make sense
ms = df_fp[df_fp["is_make_sense"] == "Yes"]

# not make sense
nms = df_fp[df_fp["is_make_sense"] == "No"]

In [221]:
ms_count = ms.groupby("template_id").count()

ms_count.tail()

,random_index,sorted_index,gender,mutant,is_make_sense,sentiment,comment
template_id,,,,,,,
5990,3,3,3,3,3,3,0
6009,3,3,3,3,3,3,0
6012,3,3,3,3,3,3,2
6041,3,3,3,3,3,3,0
6042,3,3,3,3,3,3,0


In [222]:
nms_count = nms.groupby("template_id").count()

nms_count.tail()

,random_index,sorted_index,gender,mutant,is_make_sense,sentiment,comment
template_id,,,,,,,
5280,2,2,2,2,2,2,2
5665,1,1,1,1,1,1,1
5678,1,1,1,1,1,1,1
5723,1,1,1,1,1,1,1
5829,1,1,1,1,1,1,1


In [223]:
# make sense 3 for each template only, means both template, male mutant and female mutant are make sense
mstt = ms_count[ms_count["mutant"] == 3]

# take make sense without comment
mstt = mstt[mstt["comment"] == 0]
mstt.tail()

,random_index,sorted_index,gender,mutant,is_make_sense,sentiment,comment
template_id,,,,,,,
5952,3,3,3,3,3,3,0
5990,3,3,3,3,3,3,0
6009,3,3,3,3,3,3,0
6041,3,3,3,3,3,3,0
6042,3,3,3,3,3,3,0


In [224]:
len(mstt)

153

In [225]:
# not make sense one for each template only
nmsot = nms_count[nms_count["mutant"] == 1]
nmsot.tail()

,random_index,sorted_index,gender,mutant,is_make_sense,sentiment,comment
template_id,,,,,,,
5235,1,1,1,1,1,1,1
5665,1,1,1,1,1,1,1
5678,1,1,1,1,1,1,1
5723,1,1,1,1,1,1,1
5829,1,1,1,1,1,1,1


In [226]:
len(nmsot)

36

In [227]:
# valid index
valid_index = mstt.index.values
valid_index

array([ 141,  160,  334,  400,  465,  475,  543,  555,  618,  658,  720,
        744,  798,  868,  876, 1014, 1031, 1049, 1149, 1156, 1230, 1232,
       1277, 1308, 1470, 1471, 1503, 1538, 1567, 1622, 1625, 1630, 1647,
       1672, 1695, 1712, 1736, 1887, 1914, 1946, 1986, 2027, 2076, 2086,
       2111, 2139, 2224, 2269, 2294, 2297, 2329, 2353, 2363, 2380, 2398,
       2433, 2550, 2674, 2695, 2729, 2748, 2781, 2836, 2838, 2891, 2938,
       3017, 3065, 3078, 3095, 3202, 3340, 3360, 3395, 3410, 3442, 3541,
       3560, 3575, 3605, 3613, 3697, 3770, 3771, 3854, 3872, 3910, 3913,
       4035, 4083, 4107, 4119, 4138, 4150, 4156, 4218, 4221, 4259, 4263,
       4280, 4323, 4338, 4349, 4385, 4400, 4463, 4472, 4489, 4552, 4558,
       4618, 4689, 4888, 4913, 4934, 4935, 4958, 4977, 4998, 5002, 5009,
       5056, 5126, 5155, 5219, 5245, 5282, 5305, 5310, 5321, 5324, 5362,
       5397, 5424, 5470, 5524, 5531, 5535, 5558, 5565, 5592, 5608, 5640,
       5676, 5683, 5712, 5746, 5827, 5952, 5990, 60

In [228]:
# false positive index
invalid_index = nmsot.index.values
invalid_index

array([  51,   90,  228,  629,  664,  727,  789, 1117, 1393, 2074, 2126,
       2378, 2424, 2617, 2751, 2842, 2890, 2957, 3130, 3256, 3329, 3495,
       3524, 3559, 3769, 3922, 3968, 4085, 5020, 5063, 5205, 5235, 5665,
       5678, 5723, 5829])

#### Filter data based on the template_id

In [229]:
df_fp.head()

,random_index,sorted_index,template_id,gender,mutant,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
3,540,3,1681,t,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
4,3,4,1681,m,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN


In [234]:
def is_in_array(x, arr) :
    return x in arr

def is_in_valid_index(x):
    return is_in_array(x, valid_index)

def is_in_invalid_index(x):
    return is_in_array(x, invalid_index)

# selected df using index from FP
vdf = df[df["template_id"].apply(is_in_valid_index)]
idf = df[df["template_id"].apply(is_in_invalid_index)]

In [235]:
vdf = vdf.reset_index(drop=True)
vdf.head()

,label,mutant,gender,template,name,template_id,prediction
0,1,"A Three Stooges short, this one featuring Sári...",female,"A Three Stooges short, this one featuring Shem...",Sárinka,141,1
1,1,"A Three Stooges short, this one featuring Elen...",female,"A Three Stooges short, this one featuring Shem...",Elenka,141,1
2,1,"A Three Stooges short, this one featuring Leon...",female,"A Three Stooges short, this one featuring Shem...",Leona,141,1
3,1,"A Three Stooges short, this one featuring Katu...",female,"A Three Stooges short, this one featuring Shem...",Katuška,141,1
4,1,"A Three Stooges short, this one featuring Mila...",female,"A Three Stooges short, this one featuring Shem...",Miladka,141,1


In [236]:
idf = idf.reset_index(drop=True)
idf.head()

,label,mutant,gender,template,name,template_id,prediction
0,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Sárinka,51,0
1,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Elenka,51,0
2,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Leona,51,0
3,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Katuška,51,0
4,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Miladka,51,0


In [237]:
igb = idf.groupby("template_id")
igb.count()

,label,mutant,gender,template,name,prediction
template_id,,,,,,
51,310,310,310,310,310,310
90,310,310,310,310,310,310
228,310,310,310,310,310,310
629,310,310,310,310,310,310
664,310,310,310,310,310,310
727,310,310,310,310,310,310
789,310,310,310,310,310,310
1117,310,310,310,310,310,310
1393,310,310,310,310,310,310


### Similarity Scoring using BERT

In [238]:
import pandas as pd
import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

p1 = ['Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.', 
      'ELMo is a novel way to represent words in vectors or embeddings. These word embeddings are helpful in achieving state-of-the-art (SOTA) results in several NLP tasks: NLP scientists globally have started using ELMo for various NLP tasks, both in research as well as the industry']
p2 = ['This is a dummy example here.', 
      "But here I'm trying to say something quite different from the previous sentence."]
p3 = ['Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.',
      'Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.']

p1 = ["The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Gong Li's and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Gong Li fans will be unsurprised to hear she is as stunningly beautiful as ever, giving an understated performance.",
      "The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Imre and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Imre fans will be unsurprised to hear he is as stunningly beautiful as ever, giving an understated performance."]
    
Pairs = [p1]

bc = BertClient(ip='10.4.4.55', port=5555, port_out=5556, check_version=False)
    
def similarity_scoring(pair):
    import math
    query_vec_1, query_vec_2 = bc.encode(pair)
    cosine = np.dot(query_vec_1, query_vec_2) / (np.linalg.norm(query_vec_1) * np.linalg.norm(query_vec_2))
    return 1/(1 + math.exp(-100*(cosine - 0.95)))
#         return cosine

# doc_vecs = bc.encode(questions)
# print('have reached here with questions: ', questions)

print("Start testing")

for i, p in enumerate(Pairs):
    print("Similarity of Pair {}: ".format(i+1), similarity_scoring(p))

Start testing
Similarity of Pair 1:  0.9933072283262605


/opt/conda/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


## Analysis on Invalid Mutants

In [294]:
import time
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
names_1 = []
names_2 = []
template = []
template_id = []
for i in igb.count().index.values :
    data = igb.get_group(i)
    dc = data.groupby(identifier)
    computed_group = []
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 and (k1 + "-" + k2) not in computed_group:
                computed_group.append(k1 + "-" + k2)
                computed_group.append(k2 + "-" + k1)
                for m_1, p_1, i_1, t, t_id, n_1 in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values, v1["template_id"].values, v1["name"].values) :
                    for m_2, p_2, i_2, n_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values, v2["name"].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        names_1.append(n_1)
                        names_2.append(n_2)
                        template.append(t)
                        template_id.append(t_id)
                        
end = time.time()
print("Execution time: ", end-start)

Execution time:  1.003110647201538


In [295]:
# selected pairs
sp = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "name_1": names_1, "identifier_2" : identifier_2, "name_2": names_2, "template": template, "template_id": template_id})

sp

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
0,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Roen,(Warning: Some spoilers ahead.) What an incred...,51
1,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Jeet,(Warning: Some spoilers ahead.) What an incred...,51
2,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Hagen,(Warning: Some spoilers ahead.) What an incred...,51
3,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Willow,(Warning: Some spoilers ahead.) What an incred...,51
4,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Belal,(Warning: Some spoilers ahead.) What an incred...,51
...,...,...,...,...,...,...,...,...,...,...
864895,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Cedrick,"Wow what can I say it was a good movie, very d...",5829
864896,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Rosendo,"Wow what can I say it was a good movie, very d...",5829
864897,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Quinton,"Wow what can I say it was a good movie, very d...",5829
864898,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,0,female,Alexa,male,Devon,"Wow what can I say it was a good movie, very d...",5829


In [296]:
# selected discordant pairs
sdp = sp[sp["prediction_1"] != sp["prediction_2"]]
sdp

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
0,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Roen,(Warning: Some spoilers ahead.) What an incred...,51
1,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Jeet,(Warning: Some spoilers ahead.) What an incred...,51
2,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Hagen,(Warning: Some spoilers ahead.) What an incred...,51
3,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Willow,(Warning: Some spoilers ahead.) What an incred...,51
4,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Sárinka,male,Belal,(Warning: Some spoilers ahead.) What an incred...,51
...,...,...,...,...,...,...,...,...,...,...
864894,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Svein,"Wow what can I say it was a good movie, very d...",5829
864895,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Cedrick,"Wow what can I say it was a good movie, very d...",5829
864896,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Rosendo,"Wow what can I say it was a good movie, very d...",5829
864897,"Wow what can I say it was a good movie, very d...","Wow what can I say it was a good movie, very d...",0,1,female,Alexa,male,Quinton,"Wow what can I say it was a good movie, very d...",5829


### Analysis using all data of male female from the template

In [332]:
male_similarity = []
female_similarity = []

for index, row in sdp.iterrows() :
    male_similarity.append(similarity_scoring([row["mutant_2"], row["template"]]))
    female_similarity.append(similarity_scoring([row["mutant_1"], row["template"]]))

In [333]:
sdp["male_similarity"] = male_similarity
sdp["female_similarity"] = female_similarity

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [334]:
sdp["difference"] = sdp["male_similarity"] - sdp["female_similarity"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [335]:
len(sdp[sdp["difference"] != 0]) / len(sdp)

0.220385842153476

### Analysis using a tuple of names that produce highest discordant pairs

In [336]:
sdp["combined_name"] = sdp["name_1"] + ", " + sdp["name_2"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [337]:
sdp = sdp.drop_duplicates()

In [338]:
sdp.groupby("combined_name").count().sort_values(by="template", ascending=False)

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,male_similarity,female_similarity,difference
combined_name,,,,,,,,,,,,,
"Valentīna, Axmed",25,25,25,25,25,25,25,25,25,25,25,25,25
"Valentīna, Igors",23,23,23,23,23,23,23,23,23,23,23,23,23
"Valentīna, Agnibha",23,23,23,23,23,23,23,23,23,23,23,23,23
"Valentīna, Naser",23,23,23,23,23,23,23,23,23,23,23,23,23
"Valentīna, Tyson",23,23,23,23,23,23,23,23,23,23,23,23,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Nicolle, Waabberi",2,2,2,2,2,2,2,2,2,2,2,2,2
"Dionysia, Adhipa",2,2,2,2,2,2,2,2,2,2,2,2,2
"Sylvi, Hovav",2,2,2,2,2,2,2,2,2,2,2,2,2


In [339]:
highest_pair = sdp.groupby("combined_name").get_group("Valentīna, Axmed")
highest_pair.head()

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,combined_name,male_similarity,female_similarity,difference
11072,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Valentīna,male,Axmed,(Warning: Some spoilers ahead.) What an incred...,51,"Valentīna, Axmed",0.993307,0.993307,0.000000
35097,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,1,0,female,Valentīna,male,Axmed,...and boy is the collision deafening. A femal...,90,"Valentīna, Axmed",0.974318,0.957740,0.016578
83147,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,1,0,female,Valentīna,male,Axmed,Bad news for anyone wanting to film a full-len...,629,"Valentīna, Axmed",0.993307,0.993307,0.000000
107172,"Before going any further, I have to admit that...","Before going any further, I have to admit that...",1,0,female,Valentīna,male,Axmed,"Before going any further, I have to admit that...",664,"Valentīna, Axmed",0.993307,0.993307,0.000000
131197,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,1,0,female,Valentīna,male,Axmed,Brass pictures (movies is not a fitting word f...,727,"Valentīna, Axmed",0.993307,0.993307,0.000000


In [340]:
# male
midf = idf.groupby("gender").get_group("male")

# female
fidf = idf.groupby("gender").get_group("female")
fidf.head()

,label,mutant,gender,template,name,template_id,prediction
0,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Sárinka,51,0
1,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Elenka,51,0
2,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Leona,51,0
3,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Katuška,51,0
4,0,(Warning: Some spoilers ahead.) What an incred...,female,(Warning: Some spoilers ahead.) What an incred...,Miladka,51,0


In [341]:
def get_male_is_make_sense(id):
    return mvi[mvi["template_id"] == id]["is_make_sense"].values[0]

def get_female_is_make_sense(id):
    return fvi[fvi["template_id"] == id]["is_make_sense"].values[0]

get_male_is_make_sense(3495)

'No'

In [342]:
highest_pair["male_make_sense"] = highest_pair["template_id"].apply(get_male_is_make_sense)
highest_pair["female_make_sense"] = highest_pair["template_id"].apply(get_female_is_make_sense)

highest_pair.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,combined_name,male_similarity,female_similarity,difference,male_make_sense,female_make_sense
11072,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0,1,female,Valentīna,male,Axmed,(Warning: Some spoilers ahead.) What an incred...,51,"Valentīna, Axmed",0.993307,0.993307,0.000000,Yes,No
35097,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,1,0,female,Valentīna,male,Axmed,...and boy is the collision deafening. A femal...,90,"Valentīna, Axmed",0.974318,0.957740,0.016578,No,Yes
83147,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,1,0,female,Valentīna,male,Axmed,Bad news for anyone wanting to film a full-len...,629,"Valentīna, Axmed",0.993307,0.993307,0.000000,No,Yes
107172,"Before going any further, I have to admit that...","Before going any further, I have to admit that...",1,0,female,Valentīna,male,Axmed,"Before going any further, I have to admit that...",664,"Valentīna, Axmed",0.993307,0.993307,0.000000,No,Yes
131197,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,1,0,female,Valentīna,male,Axmed,Brass pictures (movies is not a fitting word f...,727,"Valentīna, Axmed",0.993307,0.993307,0.000000,No,Yes


In [343]:
template_id = []
template = []
valid_mutant = []
invalid_mutant = []

for index,row in highest_pair.iterrows():
    template_id.append(row["template_id"])
    template.append(row["template"])
    if row["male_make_sense"] == "Yes":
        valid_mutant.append(row["mutant_1"])
        invalid_mutant.append(row["mutant_2"])
    else :
        valid_mutant.append(row["mutant_2"])
        invalid_mutant.append(row["mutant_1"])
    

In [344]:
print(len(template))
print(len(valid_mutant))
print(len(invalid_mutant))

25
25
25


In [345]:
# valid similarity
vs = []
# not valid similarity
nvs = []

for (t,v,n) in zip(template, valid_mutant, invalid_mutant):
    vs.append(similarity_scoring([t,v]))
    nvs.append(similarity_scoring([t,n]))

/opt/conda/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [346]:
d = pd.DataFrame(data={"template_id": template_id, 
                       "template": template,
                       "valid_mutant" : valid_mutant,
                       "invalid_mutant" : invalid_mutant,
                       "valid_similarity" : vs,
                       "invalid_similarity" : nvs
                      })

In [347]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity
0,51,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0.993307,0.993307
1,90,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0.974318,0.957740
2,629,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,0.993307,0.993307
3,664,"Before going any further, I have to admit that...","Before going any further, I have to admit that...","Before going any further, I have to admit that...",0.993307,0.993307
4,727,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.993307,0.993307


In [348]:
d["valid_similarity"].describe()

count    25.000000
mean      0.846676
std       0.326892
min       0.000606
25%       0.974318
50%       0.993307
75%       0.993307
max       0.993307
Name: valid_similarity, dtype: float64

In [349]:
d["invalid_similarity"].describe()

count    25.000000
mean      0.884877
std       0.287270
min       0.006415
25%       0.993307
50%       0.993307
75%       0.993307
max       0.993307
Name: invalid_similarity, dtype: float64

In [350]:
d["difference"] = d["valid_similarity"] - d["invalid_similarity"]

In [351]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
0,51,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0.993307,0.993307,0.000000
1,90,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0.974318,0.957740,0.016578
2,629,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,Bad news for anyone wanting to film a full-len...,0.993307,0.993307,0.000000
3,664,"Before going any further, I have to admit that...","Before going any further, I have to admit that...","Before going any further, I have to admit that...",0.993307,0.993307,0.000000
4,727,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.993307,0.993307,0.000000


In [352]:
d["difference"].describe()

count    25.000000
mean     -0.038201
std       0.120172
min      -0.507936
25%       0.000000
50%       0.000000
75%       0.000000
max       0.016578
Name: difference, dtype: float64

In [353]:
d[d["difference"] != 0]

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
1,90,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,...and boy is the collision deafening. A femal...,0.974318,0.957740,0.016578
5,789,Charlie Chaplin's Little Tramp or Little Man c...,Valentīna Little Tramp or Little Man characte...,Axmed Little Tramp or Little Man character wi...,0.020969,0.008676,0.012293
9,2617,"I'm sorry, but ""Star Wars Episode 1"" did not d...","I'm sorry, but ""Star Wars Episode 1"" did not d...","I'm sorry, but ""Star Wars Episode 1"" did not d...",0.086336,0.427602,-0.341266
10,2842,"In this episode, a man and his dog go 'coon hu...","In this episode, a man and her dog go 'coon hu...","In this episode, a man and his dog go 'coon hu...",0.974874,0.993307,-0.018433
11,2957,It's fun to watch a young James Cagney doing h...,It's fun to watch Valentīna doing her thing. S...,It's fun to watch Axmed doing his thing. He pl...,0.000606,0.006415,-0.005808
18,5020,This movie is about a man who likes to blow hi...,This movie is about a man who likes to blow he...,This movie is about a man who likes to blow hi...,0.967332,0.993307,-0.025975
19,5063,"This movie is scary at times, perhaps no more ...","This movie is scary at times, perhaps no more ...","This movie is scary at times, perhaps no more ...",0.828796,0.913274,-0.084478
23,5678,"When Nathaniel Kahn embarked into this voyage,...","When Valentīna embarked into this voyage, she ...","When Axmed embarked into this voyage, he hardl...",0.427456,0.935392,-0.507936


In [354]:
d[d["template_id"] == 5678]["invalid_mutant"].values

array(["When Axmed embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child. Axmed was the father. He was an architect's architect. His designs were perhaps too complex, as he tried to create buildings that didn't conform with trends popular at that time. It is ironic that he never achieved the fame that came so easy to some of his contemporaries. He had a vision and he never strayed from it. We can see characteristics of his unique style in the buildings he left behind as a legacy to humanity. Every one of his creations are unique in that they don't imitate works from other architects. Axmed Axmed life was rather complicated. He was married, yet he had affairs with two of his assistants that produced a girl and a boy, besides the legitimate daughter he had with his wife. As a boy, Axmed life was lived in a secluded area, away from his father, who only visited late at night. Ax

#### Percentage of filtered mutants

In [355]:
len(d[d["difference"] != 0]) / len(d)

0.32

## Analysis on Valid Mutants

In [356]:
import time
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
names_1 = []
names_2 = []
template = []
template_id = []
for i in vgb.count().index.values :
    data = vgb.get_group(i)
    dc = data.groupby(identifier)
    computed_group = []
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 and (k1 + "-" + k2) not in computed_group:
                computed_group.append(k1 + "-" + k2)
                computed_group.append(k2 + "-" + k1)
                for m_1, p_1, i_1, t, t_id, n_1 in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values, v1["template_id"].values, v1["name"].values) :
                    for m_2, p_2, i_2, n_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values, v2["name"].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        names_1.append(n_1)
                        names_2.append(n_2)
                        template.append(t)
                        template_id.append(t_id)
                        
end = time.time()
print("Execution time: ", end-start)

Execution time:  4.353043079376221


In [357]:
# selected pairs
sp = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "name_1": names_1, "identifier_2" : identifier_2, "name_2": names_2, "template": template, "template_id": template_id})

sp

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
0,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Roen...",1,1,female,Sárinka,male,Roen,"A Three Stooges short, this one featuring Shem...",141
1,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Jeet...",1,1,female,Sárinka,male,Jeet,"A Three Stooges short, this one featuring Shem...",141
2,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Hage...",1,1,female,Sárinka,male,Hagen,"A Three Stooges short, this one featuring Shem...",141
3,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Will...",1,1,female,Sárinka,male,Willow,"A Three Stooges short, this one featuring Shem...",141
4,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Bela...",1,0,female,Sárinka,male,Belal,"A Three Stooges short, this one featuring Shem...",141
...,...,...,...,...,...,...,...,...,...,...
3675820,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,1,female,Alexa,male,Cedrick,"this movie has a great chase scene, if you lis...",6042
3675821,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,1,female,Alexa,male,Rosendo,"this movie has a great chase scene, if you lis...",6042
3675822,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,1,female,Alexa,male,Quinton,"this movie has a great chase scene, if you lis...",6042
3675823,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,1,female,Alexa,male,Devon,"this movie has a great chase scene, if you lis...",6042


In [358]:
# selected discordant pairs
sdp = sp[sp["prediction_1"] != sp["prediction_2"]]
sdp

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
4,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Bela...",1,0,female,Sárinka,male,Belal,"A Three Stooges short, this one featuring Shem...",141
12,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Shah...",1,0,female,Sárinka,male,Shahab,"A Three Stooges short, this one featuring Shem...",141
25,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Hova...",1,0,female,Sárinka,male,Hovav,"A Three Stooges short, this one featuring Shem...",141
29,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Nahs...",1,0,female,Sárinka,male,Nahshon,"A Three Stooges short, this one featuring Shem...",141
30,"A Three Stooges short, this one featuring Sári...","A Three Stooges short, this one featuring Arag...",1,0,female,Sárinka,male,Araguarino,"A Three Stooges short, this one featuring Shem...",141
...,...,...,...,...,...,...,...,...,...,...
3675186,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,0,female,Dódi,male,Igors,"this movie has a great chase scene, if you lis...",6042
3675341,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,0,female,Iboyka,male,Igors,"this movie has a great chase scene, if you lis...",6042
3675496,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,0,female,Zsófia,male,Igors,"this movie has a great chase scene, if you lis...",6042
3675651,"this movie has a great chase scene, if you lis...","this movie has a great chase scene, if you lis...",1,0,female,Ágota,male,Igors,"this movie has a great chase scene, if you lis...",6042


### Analysis using all data of male female from the template

In [359]:
male_similarity = []
female_similarity = []

for index, row in sdp.iterrows() :
    male_similarity.append(similarity_scoring([row["mutant_2"], row["template"]]))
    female_similarity.append(similarity_scoring([row["mutant_1"], row["template"]]))

/opt/conda/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [360]:
sdp["male_similarity"] = male_similarity
sdp["female_similarity"] = female_similarity

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [361]:
sdp["difference"] = sdp["male_similarity"] - sdp["female_similarity"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [362]:
len(sdp[sdp["difference"] != 0]) / len(sdp)

0.14383370656524458

### Analysis using a tuple of names that produce highest discordant pairs

In [269]:
sdp["combined_name"] = sdp["name_1"] + ", " + sdp["name_2"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [270]:
sdp = sdp.drop_duplicates()

In [271]:
sdp.groupby("combined_name").count().sort_values(by="template", ascending=False)

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id
combined_name,,,,,,,,,,
"Valentīna, Axmed",79,79,79,79,79,79,79,79,79,79
"Daryell-Hannah, Cedrick",76,76,76,76,76,76,76,76,76,76
"Snandulija, Cedrick",75,75,75,75,75,75,75,75,75,75
"Sahra, Cedrick",74,74,74,74,74,74,74,74,74,74
"Priyadarshini, Cedrick",74,74,74,74,74,74,74,74,74,74
...,...,...,...,...,...,...,...,...,...,...
"Elenka, Elemer",3,3,3,3,3,3,3,3,3,3
"Lucila, Vuand",3,3,3,3,3,3,3,3,3,3
"Rawan, Meri̇ç",3,3,3,3,3,3,3,3,3,3


In [272]:
highest_pair = sdp.groupby("combined_name").get_group("Valentīna, Axmed")
highest_pair.head()

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,combined_name
11072,"A Three Stooges short, this one featuring Vale...","A Three Stooges short, this one featuring Axme...",1,0,female,Valentīna,male,Axmed,"A Three Stooges short, this one featuring Shem...",141,"Valentīna, Axmed"
35097,Valentīna dreams of going to Mars. She eventu...,Axmed dreams of going to Mars. He eventually ...,1,0,female,Valentīna,male,Axmed,A dedicated Russian Scientist dreams of going ...,160,"Valentīna, Axmed"
59122,After reading a biography on the last Russian ...,After reading a biography on the last Russian ...,1,0,female,Valentīna,male,Axmed,After reading a biography on the last Russian ...,334,"Valentīna, Axmed"
83147,All those that identify this as a simple rip-o...,All those that identify this as a simple rip-o...,1,0,female,Valentīna,male,Axmed,All those that identify this as a simple rip-o...,400,"Valentīna, Axmed"
107172,An obscure horror show filmed in the Everglade...,An obscure horror show filmed in the Everglade...,1,0,female,Valentīna,male,Axmed,An obscure horror show filmed in the Everglade...,465,"Valentīna, Axmed"


In [275]:
highest_pair["male_make_sense"] = ["Yes"] * len(highest_pair["template_id"])
highest_pair["female_make_sense"] = ["Yes"] * len(highest_pair["template_id"])

highest_pair.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,name_1,identifier_2,name_2,template,template_id,combined_name,male_make_sense,female_make_sense
11072,"A Three Stooges short, this one featuring Vale...","A Three Stooges short, this one featuring Axme...",1,0,female,Valentīna,male,Axmed,"A Three Stooges short, this one featuring Shem...",141,"Valentīna, Axmed",Yes,Yes
35097,Valentīna dreams of going to Mars. She eventu...,Axmed dreams of going to Mars. He eventually ...,1,0,female,Valentīna,male,Axmed,A dedicated Russian Scientist dreams of going ...,160,"Valentīna, Axmed",Yes,Yes
59122,After reading a biography on the last Russian ...,After reading a biography on the last Russian ...,1,0,female,Valentīna,male,Axmed,After reading a biography on the last Russian ...,334,"Valentīna, Axmed",Yes,Yes
83147,All those that identify this as a simple rip-o...,All those that identify this as a simple rip-o...,1,0,female,Valentīna,male,Axmed,All those that identify this as a simple rip-o...,400,"Valentīna, Axmed",Yes,Yes
107172,An obscure horror show filmed in the Everglade...,An obscure horror show filmed in the Everglade...,1,0,female,Valentīna,male,Axmed,An obscure horror show filmed in the Everglade...,465,"Valentīna, Axmed",Yes,Yes


In [276]:
template_id = []
template = []
valid_mutant = []
invalid_mutant = []

for index,row in highest_pair.iterrows():
    template_id.append(row["template_id"])
    template.append(row["template"])
    if row["male_make_sense"] == "Yes":
        valid_mutant.append(row["mutant_1"])
        invalid_mutant.append(row["mutant_2"])
    else :
        valid_mutant.append(row["mutant_2"])
        invalid_mutant.append(row["mutant_1"])
    

In [277]:
print(len(template))
print(len(valid_mutant))
print(len(invalid_mutant))

79
79
79


In [278]:
# valid similarity
vs = []
# not valid similarity
nvs = []

for (t,v,n) in zip(template, valid_mutant, invalid_mutant):
    vs.append(similarity_scoring([t,v]))
    nvs.append(similarity_scoring([t,n]))

/opt/conda/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [279]:
d = pd.DataFrame(data={"template_id": template_id, 
                       "template": template,
                       "valid_mutant" : valid_mutant,
                       "invalid_mutant" : invalid_mutant,
                       "valid_similarity" : vs,
                       "invalid_similarity" : nvs
                      })

In [280]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity
0,141,"A Three Stooges short, this one featuring Shem...","A Three Stooges short, this one featuring Vale...","A Three Stooges short, this one featuring Axme...",0.271096,0.687829
1,160,A dedicated Russian Scientist dreams of going ...,Valentīna dreams of going to Mars. She eventu...,Axmed dreams of going to Mars. He eventually ...,0.118473,0.444908
2,334,After reading a biography on the last Russian ...,After reading a biography on the last Russian ...,After reading a biography on the last Russian ...,0.993307,0.993307
3,400,All those that identify this as a simple rip-o...,All those that identify this as a simple rip-o...,All those that identify this as a simple rip-o...,0.993307,0.993307
4,465,An obscure horror show filmed in the Everglade...,An obscure horror show filmed in the Everglade...,An obscure horror show filmed in the Everglade...,0.993307,0.993307


In [281]:
d["valid_similarity"].describe()

count    79.000000
mean      0.898500
std       0.253830
min       0.002779
25%       0.993307
50%       0.993307
75%       0.993307
max       0.993307
Name: valid_similarity, dtype: float64

In [282]:
d["invalid_similarity"].describe()

count    79.000000
mean      0.920575
std       0.205141
min       0.095890
25%       0.993307
50%       0.993307
75%       0.993307
max       0.993307
Name: invalid_similarity, dtype: float64

In [283]:
d["difference"] = d["valid_similarity"] - d["invalid_similarity"]

In [284]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
0,141,"A Three Stooges short, this one featuring Shem...","A Three Stooges short, this one featuring Vale...","A Three Stooges short, this one featuring Axme...",0.271096,0.687829,-0.416732
1,160,A dedicated Russian Scientist dreams of going ...,Valentīna dreams of going to Mars. She eventu...,Axmed dreams of going to Mars. He eventually ...,0.118473,0.444908,-0.326435
2,334,After reading a biography on the last Russian ...,After reading a biography on the last Russian ...,After reading a biography on the last Russian ...,0.993307,0.993307,0.000000
3,400,All those that identify this as a simple rip-o...,All those that identify this as a simple rip-o...,All those that identify this as a simple rip-o...,0.993307,0.993307,0.000000
4,465,An obscure horror show filmed in the Everglade...,An obscure horror show filmed in the Everglade...,An obscure horror show filmed in the Everglade...,0.993307,0.993307,0.000000


In [285]:
d["difference"].describe()

count    79.000000
mean     -0.022075
std       0.128508
min      -0.503780
25%       0.000000
50%       0.000000
75%       0.000000
max       0.593069
Name: difference, dtype: float64

In [286]:
d[d["difference"] != 0]

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
0,141,"A Three Stooges short, this one featuring Shem...","A Three Stooges short, this one featuring Vale...","A Three Stooges short, this one featuring Axme...",0.271096,0.687829,-0.416732
1,160,A dedicated Russian Scientist dreams of going ...,Valentīna dreams of going to Mars. She eventu...,Axmed dreams of going to Mars. He eventually ...,0.118473,0.444908,-0.326435
10,798,Chris Morris' Brass Eye TV series had always g...,Valentīna Brass Eye TV series had always gene...,Axmed Brass Eye TV series had always generate...,0.085921,0.581022,-0.495101
11,876,De Dominee is based on the life of a real dutc...,Valentīna is based on the life of a real dutc...,Axmed is based on the life of a real dutch ga...,0.797837,0.858471,-0.060634
18,1567,I can understand how Barney can be annoying to...,I can understand how Valentīna can be annoying...,I can understand how Axmed can be annoying to ...,0.012779,0.106852,-0.094073
31,2297,I think that just sums up this film. Watch it ...,I think that just sums up this film. Watch it ...,I think that just sums up this film. Watch it ...,0.175683,0.214833,-0.039150
42,3095,"Kane is a killer named Jacob Goodnight, he liv...","Valentīna is a killer named Jacob Goodnight, ...","Axmed is a killer named Jacob Goodnight, he l...",0.878445,0.973462,-0.095017
43,3202,Lucio Fulci was one of the most prolific Itali...,Valentīna was one of the most prolific Italia...,Axmed was one of the most prolific Italian di...,0.002779,0.095890,-0.093112
47,3910,Since others have complained about putting She...,Since others have complained about putting Val...,Since others have complained about putting Axm...,0.265522,0.769301,-0.503780
51,4221,The best bit in the film was when Alan pulled ...,The best bit in the film was when Valentīna pu...,The best bit in the film was when Axmed pulled...,0.440356,0.757254,-0.316899


#### Number of Filtered Mutants

In [287]:
len(d[d["difference"] != 0]) / len(d)

0.17721518987341772